# 의사록 hwp 파일 크롤링 (final)

In [1]:
import pandas as pd
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.common.by import By
import time

def get_hwp_data(startyear, startmonth, endyear, endmonth):
    driver = webdriver.Chrome()
    
    result = []

    try:
        while startyear <= endyear:
            url = f'https://www.bok.or.kr/portal/singl/crncyPolicyDrcMtg/listYear.do?mtgSe=A&menuNo=200755&pYear={startyear}#content'
            driver.get(url)
            time.sleep(3)
            
            tr_tags = driver.find_elements(By.CSS_SELECTOR, '#tableId > tbody > tr')
            
            for tr_tag in tr_tags:
                day = tr_tag.find_element(By.CSS_SELECTOR,'th').text
                month = int(day[:2])

                if startyear == 2005 and month < startmonth:
                    continue
                if endyear == 2012 and month > endmonth:
                    break

                td_tag = tr_tag.find_elements(By.CSS_SELECTOR,'td')[2]
                if startyear == 2006 or (startyear == 2007) & (month == 1):
                    li_tag = td_tag.find_elements(By.CSS_SELECTOR,'div.fileGroupSet>div.fileGoupBox>ul>li')[1]
                    a_tag = li_tag.find_element(By.CSS_SELECTOR,'a')
                    dlink = a_tag.get_attribute('href')
                    title = (a_tag.get_attribute('title'))[:-4]
                else:
                    a_tag = td_tag.find_element(By.CSS_SELECTOR,'div.fileGroupSet>div.fileGoupBox>ul>li>a')
                    dlink = a_tag.get_attribute('href')
                    if (startyear == 2008) & (month >= 11) or startyear >= 2009:
                        title = f'{startyear}년+{(a_tag.get_attribute("title"))[:-4]}'
                    elif (startyear == 2008) & (month == 10):
                        title = '2008년+제20차+금통위+의사록'
                    elif (startyear == 2007) and (month == 2 or month == 4 or month == 7):
                        title = (a_tag.get_attribute('title'))[:-8]
                    else:
                        title = (a_tag.get_attribute('title'))[:-4]
                result.append(dict(
                    year = startyear,
                    day = day,
                    dlink = dlink,
                    title = title.replace('+', ' ')
                ))

            startyear += 1
            
    finally:
        driver.quit()

    return result

def convert_to_datetime(df):
    # 'day' 컬럼에서 "월"과 "일" 부분 제거
    df['day'] = df['day'].str.replace('월', '').str.replace('일', '').str.split('(').str[0].str.strip()
    
    # 'year'와 'day'를 합쳐서 새로운 컬럼 'date' 생성
    df['date'] = df['year'].astype(str) + ' ' + df['day']
    
    # 'date' 컬럼을 datetime 타입으로 변환
    df['date'] = pd.to_datetime(df['date'], format='%Y %m %d')
    
    # 'year'와 'day' 컬럼 제거
    df = df.drop(columns=['year', 'day'])
    
    # 열 순서를 변경하여 'date'가 먼저 오도록 설정
    df = df[['date', 'title', 'dlink']]
    
    return df

data = get_hwp_data(2005, 5, 2017, 12)
df = pd.DataFrame(data, columns=['year', 'day', 'title', 'dlink'])

df = convert_to_datetime(df)


In [2]:
#다운로드 파일에 저장하기

from selenium import webdriver
from selenium.webdriver.common.by import By


driver = webdriver.Chrome()

for i in df['dlink']:
    driver.get(i)

time.sleep(20)

driver.quit()

In [3]:
#filename 리스트 만들기

import os

path = "C:/Users/kwkwo/Downloads/"
all_filenames = os.listdir(path)
hwp_files = []

for filename in all_filenames:
    if filename[-4:] == ".hwp":
        hwp_files.append(filename)

In [4]:
import olefile
import zlib
import struct
import os
import re

# 다운로드 폴더 경로 (Windows에서는 기본적으로 사용자 폴더에 있음)
download_folder = os.path.join(os.path.expanduser("~"), "Downloads")
 
# .hwp 파일 목록 가져오기
hwp_files = [os.path.join(download_folder, f) for f in os.listdir(download_folder) if f.endswith('.hwp')]

# '통화정책방향〉' ~ '(４) 심의결과' 사이의 내용만 가져오는 함수
def extract_text_after_keyword(text):
    start_keyword = "통화정책방향〉"
    end_keyword = "(４) 심의결과"
    
    if start_keyword in text:
        # '통화정책방향〉' 이후의 텍스트만 추출
        extracted_text = text.split(start_keyword, 1)[1].strip()
        
        if end_keyword in extracted_text:
            # '(４) 심의결과' 이전의 텍스트만 반환
            extracted_text = extracted_text.split(end_keyword, 1)[0].strip()
        
        return extracted_text
    
    # 키워드가 없으면 빈 문자열 반환
    return ""

def get_hwp_text(filename):
    f = olefile.OleFileIO(filename)
    dirs = f.listdir()
    # print(dirs)

    # HWP 파일 검증
    if ["FileHeader"] not in dirs or \
            ["\x05HwpSummaryInformation"] not in dirs:
        raise Exception("Not Valid HWP.")

    # 문서 포맷 압축 여부 확인
    header = f.openstream("FileHeader")
    header_data = header.read()
    is_compressed = (header_data[36] & 1) == 1

    # Body Sections 불러오기
    nums = []
    for d in dirs:
        if d[0] == "BodyText":
            nums.append(int(d[1][len("Section"):]))
    sections = ["BodyText/Section" + str(x) for x in sorted(nums)]

    # 전체 text 추출
    text = ""
    for section in sections:
        bodytext = f.openstream(section)
        data = bodytext.read()
        if is_compressed:
            unpacked_data = zlib.decompress(data, -15)
        else:
            unpacked_data = data

        # 각 Section 내 text 추출
        section_text = ""
        i = 0
        size = len(unpacked_data)
        while i < size:
            header = struct.unpack_from("<I", unpacked_data, i)[0]
            rec_type = header & 0x3ff
            rec_len = (header >> 20) & 0xfff

            if rec_type in [67]:
                rec_data = unpacked_data[i + 4:i + 4 + rec_len]
                section_text += rec_data.decode('utf-16')
                section_text += "\n"

            i += 4 + rec_len

        text += section_text
        text += "\n"

    return text

# 리스트 초기화
contents = []

for hwp in hwp_files:
    # print(hwp)
    text = get_hwp_text(hwp)
    contents.append(text)

# 텍스트 정제
extracted_contents = [extract_text_after_keyword(text) for text in contents]

In [7]:
df['content'] = extracted_contents

# df를 db에 저장

In [19]:
import json
import pymysql
from datetime import datetime

db = pymysql.connect(
    host='paper-project-1.cne82okq8qfy.us-east-2.rds.amazonaws.com', 
    port=3306, 
    user="heeae", 
    passwd="0000", 
    db="PAPER_PROJECT", 
    charset="utf8"
)

cursor = db.cursor()

#SQL에 데이터 넣기 (수정: json 키 "content" > "keyword")
sql = "INSERT INTO text_analysis (document_type, date, title, sentences) VALUES (%s, %s, %s, %s);"
for i, row in df.iterrows():
    sentences_json = json.dumps({"keyword": row["contents"]})
    cursor.execute(sql, ('mpc_minutes', row["date"], row["title"], sentences_json))
  
 
db.commit() 
db.close()

### sentences 행 {'keyword':"파일 단위 text"} > {'keyword':[[문장1],[문장2],...]}

In [2]:
import json
import pymysql

# 데이터베이스에 연결
db = pymysql.connect(
    host='paper-project-1.cne82okq8qfy.us-east-2.rds.amazonaws.com', 
    port=3306, 
    user="heeae", 
    passwd="0000", 
    db="PAPER_PROJECT", 
    charset="utf8"
)
cursor = db.cursor()

# 테이블에서 데이터를 선택
sql = "SELECT id, sentences from text_analysis WHERE document_type = 'mpc_minutes';"
cursor.execute(sql)
records = cursor.fetchall()

for record in records:
    # id와 현재 JSON 데이터를 추출
    record_id = record[0]
    current_json = record[1]
    
    # 현재 JSON 데이터를 로드
    data = json.loads(current_json)
    
    # 'content'를 문장으로 나누고 2차원 리스트로 변환
    sentences = data['keyword'].split('\r\n')
    sents_as_listlist = [[sent] for sent in sentences]

    # 변환된 2차원 리스트를 content에 업데이트
    data['keyword'] = sents_as_listlist

    # 업데이트된 딕셔너리를 JSON 문자열로 변환
    updated_json = json.dumps(data)
    
    # 변환된 JSON 문자열을 데이터베이스에 업데이트
    update_sql = "UPDATE text_analysis SET sentences = %s WHERE id = %s"
    cursor.execute(update_sql, (updated_json, record_id))

# 데이터베이스에 변경 사항을 커밋
db.commit()

# 커서와 데이터베이스 연결 종료
cursor.close()
db.close()

KeyboardInterrupt: 